## Loading the Dataset

In [1]:
# Code to read csv file into Colaboratory:
# !pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
# Loading the necessary packages
import pandas as pd
import numpy as np
from google.colab import files

In [3]:
# link for january
link = 'https://drive.google.com/file/d/1KrLtMptObQUhqqYaic0QCIDo7GhuHf4F/view?usp=share_link' # The shareable link

# getting the id part of the link
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('january_2019.csv') 
 
january = pd.read_csv('january_2019.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (292) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# link for february
link = 'https://drive.google.com/file/d/1HUyOCktP39i4-40uod7IiODqYQ4kvBrC/view?usp=share_link' # The shareable link

# getting the id part of the link
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('february_2019.csv') 
 
february = pd.read_csv('february_2019.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (196,292) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# link for march
link = 'https://drive.google.com/file/d/1KrLtMptObQUhqqYaic0QCIDo7GhuHf4F/view?usp=share_link' # The shareable link

# getting the id part of the link
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('march_2019.csv') 
 
march = pd.read_csv('march_2019.csv')

In [6]:
# link for april
link = 'https://drive.google.com/file/d/1DAmdYfR08EMjH2vWuD_OE6tQqOvDsKa9/view?usp=share_link' # The shareable link

# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('april_2019.csv') 
 
april = pd.read_csv('april_2019.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (95,196,292) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
# Uniting the sample
df = [january, february, march, april]
df = pd.concat(df).reset_index()

In [8]:
# looking at it's size
df.shape

(166030, 299)

## Data Cleaning

In [9]:
# Dropping repeated columns
df.drop(["Unnamed: 0", "MES_y.1", "MES_y", "DIRECTORIO_x", "DIRECTORIO_y", "DIRECTORIO_x.1", "DIRECTORIO_y.1", "SECUENCIA_P_x", "SECUENCIA_P_y", "SECUENCIA_P_x.1", "SECUENCIA_P_y.1", "ORDEN_x", "ORDEN_y", "ORDEN_x.1", "ORDEN_y.1", "HOGAR_x", "HOGAR_y", "HOGAR_x.1", "HOGAR_y.1", "DPTO_x", "DPTO_y", "DPTO_x.1", "DPTO_y.1", "base_x", "base_y", "base_x.1", "base_y.1", "REGIS_y", "REGIS_y.1"], axis=1, inplace=True)

### Getting rid of 98, code for invalid observations

In [10]:
# Create a dataframe which is a table wich counts the amount of 98 or "98" in each column
df_1 = df[(df == 98)|(df == "98")].count().sort_values()

In [11]:
# Create a dataframe which only keeps the columns that have a positive amount of invalid observations
a = pd.DataFrame(df_1[df_1 != 0])

# create a list with the names of the columns for which the condition holds
a = a.index.array 
# a is 49 columns

In [12]:
# drops columns that are in a
for i in df.columns.values: # for every column in the df
  if i in a: # if the name of the column is inside a
    df.drop(i, axis = 1,inplace = True) # delete it
  else: # if not
    continue # keep it

In [13]:
df.shape

(166030, 221)

### Getting rid of NAs

In [14]:
# Erasing the rows that have NAs in our dependant variable
df = df.dropna(subset=['P6440'])

In [15]:
# Create a dataframe which reflects the amount of nas for each column
df_2 = df.isna().sum()

# storing the columns that have a positive amount of nas
df_2 = df_2[df_2 != 0] 

len(df_2) # only AREA with 10419

1

In [16]:
# dropping the COLUMNS with nas
df = df.dropna(axis='columns')

In [17]:
df.shape

(166030, 220)

### Getting rid of empty values

In [18]:
# Using the replace function to replace all empty values for the value 1000
df_3 = df.replace(" ", 1000, regex = True)

# Create a dataframe which is a table wich counts the amount of 1000 in each column
df_3 = df_3[(df_3 == 1000)].count().sort_values()

df_3

P388            0
P6070           0
P6050           0
P6020           0
P6016           0
            ...  
P6310      166030
P6300      166030
P6280      166030
P7050S1    166030
P6330      166030
Length: 220, dtype: int64

In [19]:
# filtering the dtaframe for those columns which had a positive amount of empty values
c = pd.DataFrame(df_3[df_3 != 0])

# creating a list with those columns
c = c.index.array
# c is 49 columns
c

<PandasArray>
[    'ESC',   'P6920', 'P6030S1', 'P6030S3',   'P6980', 'INGLABO',   'P6100',
 'P6810S1',   'P7028',   'P1880',
 ...
   'P6351',   'P6350',   'P6340',   'P6320', 'P6310S1',   'P6310',   'P6300',
   'P6280', 'P7050S1',   'P6330']
Length: 159, dtype: object

In [20]:
# drops columns that are in c
for i in df.columns.values: # for every column in the df
  if i in c: # if the name of the column is inside c
    df.drop(i, axis = 1,inplace = True) # delete it
  else: # if not
    continue # keep it

## Understanding our Dependant Variable

In [21]:
# Creating a copy of our dataset
df_4 = df

# Replace all 2 for 0
df_5 = df_4["P6440"].replace(2, 0, regex = True)

# Separating the dependant variable
print(f"{len(df_5[df_5 == 1])} do not have a contract, while {len(df_5[df_5 == 0])} have it")

91496 do not have a contract, while 74534 have it


In [22]:
# Defining dependant variable
y = df['P6440']

# setting the rest of the database
X = df.drop(['P6440'],axis=1)

## Models

In [23]:
categorical = ['P388', 'P6790', 'P6870', 'P6880', 'P6915', 'P6990',
       'P7020', 'P7040', 'P1881', 'P7090', 'P7130', 'P7170S1', 'P7170S6',
       'P514', 'P515', 'P7240', 'P6020', 'P6050', 
       'P6125', 'P6160', 'P6170', 'P6210', 'P6240', 'P7480S1',
       'P7480S3', 'P7480S4', 'P7480S5', 'P7480S6', 'P7480S7', 'P7480S8',
       'P7480S9', 'P7480S10', 'P7480S11', 'P7505', 'MES', 'DPTO', 'base',
       'CLASE']

numerical = ["P6790"]

In [24]:
# All packages 

# Line to split the code
from sklearn.model_selection import train_test_split

# Lines for OneHotEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

# Lines for Standardscaler
from sklearn.preprocessing import StandardScaler

# Line for Logistic Regression
from sklearn.linear_model import LogisticRegression

# Line for F1 Score
from sklearn.metrics import f1_score

# Line for Matthews Correlation Coefficient
from sklearn.metrics import matthews_corrcoef

# Line for cross validation
from sklearn.model_selection import cross_validate

#Line for confusion matrix
from sklearn.metrics import ConfusionMatrixDisplay

# Line for Decision Tree 
from sklearn.tree import DecisionTreeClassifier

# Line for XGBoost
import xgboost as xgb

### Logistic Regression

In [25]:
# Setting up the training and testing set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, random_state=537)

In [26]:
# We use onehotencoder to encode our categorical varibales into binary columns
ohe = OneHotEncoder( handle_unknown='ignore')
se = StandardScaler()

# Preprocess 
preprocessor = ColumnTransformer(
    transformers = [
        ("num",StandardScaler(),numerical),
        ("cat",ohe,categorical),
    ]
)

In [27]:
# Create a pipeline
pipe_log = Pipeline([
    ('preproc', preprocessor),
    ('model', LogisticRegression(max_iter= 1000,
                                 solver = 'lbfgs'))
    ])

# Fit
y_pred_log = pipe_log.fit(X_train, y_train).predict(X_train)

In [28]:
# Using our log model to predict the testing set
y_pred_log = pipe_log.fit(X_train, y_train).predict(X_test)

In [29]:
# Calculating the F1 Score
f1_log = f1_score(y_pred_log, y_test, average="macro")
f1_log

0.9091585458908498

In [30]:
# Calculating the MCC
mcc_log = matthews_corrcoef(y_pred_log, y_test)
mcc_log

0.8188134469578523

### Decision Trees

In [31]:
# Setting up the training and testing set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, random_state=573)

In [32]:
# We use onehotencoder to encode our categorical varibales into binary columns
ohe = OneHotEncoder(handle_unknown='ignore')
se = StandardScaler()

# Preprocess 
preprocessor = ColumnTransformer(
    transformers = [
        ("num",StandardScaler(),numerical),
        ("cat",ohe,categorical),
    ]
)

In [33]:
# Create a pipeline
pipe_tree = Pipeline([
    ('preproc', preprocessor),
    ('model', DecisionTreeClassifier())
    ])

# # Fit
y_pred_tree = pipe_tree.fit(X_train, y_train).predict(X_train)

In [34]:
# Using our tree model to predict the testing set
y_pred_tree = pipe_tree.fit(X_train, y_train).predict(X_test)

In [35]:
f1_tree = f1_score(y_pred_tree, y_test, average="macro")
f1_tree

0.9585640290813051

In [36]:
mcc_tree = matthews_corrcoef(y_pred_tree, y_test)
mcc_tree

0.9171316996779082

### XGBoost

In [37]:
# Setting up the training and testing set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, random_state=573)

In [38]:
# We use onehotencoder to encode our categorical varibales into binary columns
ohe = OneHotEncoder(handle_unknown='ignore')
se = StandardScaler()

# Preprocess 
preprocessor = ColumnTransformer(
    transformers = [
        ("num",StandardScaler(),numerical),
        ("cat",ohe,categorical),
    ]
)

In [39]:
# Create a pipeline
pipe_xg = Pipeline([
    ('preproc', preprocessor),
    ('model', xgb.XGBClassifier())
    ])

In [40]:
# Using our xg model to predict the testing set
y_pred_xg = pipe_xg.fit(X_train, y_train).predict(X_test)

In [41]:
f1_xg = f1_score(y_pred_xg, y_test, average="macro")
f1_xg

0.907046737081439

In [42]:
mcc_xg = matthews_corrcoef(y_pred_xg, y_test)
mcc_xg

0.8141497132791358